In [2]:
import numpy as np
import pandas as pd
from astropy.io import fits
import glob

import matplotlib.pyplot as plt
%matplotlib notebook

The following query was used to select **obvious stars** from the Gaia-PS1 crossmatch table: 

    select ps1.*,
    sqrt(power(pmra,2) + power(pmdec,2)) as pm, 
    sqrt(  power(pmra,2)*power(pmra_error, 2)/(power(pmra,2) + power(pmdec,2)) 
	 + power(pmdec,2)*power(pmdec_error, 2)/(power(pmra,2) + power(pmdec,2)) 
	 + 2*pmra*pmdec/(power(pmra,2) + power(pmdec,2))*pmra_pmdec_corr*pmra_error*pmdec_error) as pm_unc, 
    gaia.parallax_over_error, gaia.phot_g_mean_mag, gaia.b
    from gaiadr2.panstarrs1_best_neighbour as ps1 
    inner join gaiadr2.gaia_source as gaia
    on ps1.source_id = gaia.source_id
    where gaia.astrometric_params_solved > 3
    and gaia.dec between -10 and -5
    and phot_bp_rp_excess_factor < 1.3+0.06*power(phot_bp_mean_mag-phot_rp_mean_mag,2)
    AND phot_bp_rp_excess_factor > 1.0+0.015*power(phot_bp_mean_mag-phot_rp_mean_mag,2)
    AND astrometric_chi2_al/(astrometric_n_good_obs_al-5) < 1.44*greatest(1,exp(-0.4*(phot_g_mean_mag-19.5)))
    order by ps1.original_ext_source_id

where the above query produced the file `dec_neg10-result.fits` and the declination range was adjusted to create all the other files.

## Create the loop to put all the objid into a single file

In [5]:
gaia_dir = "/Users/adamamiller/Desktop/PS1_fits/gaia_stars/"
gaia_files = glob.glob(gaia_dir + "tmp_dec*fits")
for file_num, gf in enumerate(gaia_files):
    gdat = fits.getdata(gf)
    parallax_stars = np.where( (gdat['parallax_over_error'] >= 8) & 
                               (gdat['number_of_neighbours'] == 1) & 
                               (gdat['number_of_mates'] == 0))
    pm_stars = np.where((gdat['pm']/gdat['pm_unc'] >= 7.5) & 
                         (gdat['number_of_neighbours'] == 1) & 
                         (gdat['number_of_mates'] == 0))
    tmp_df = pd.DataFrame(gdat['original_ext_source_id'][parallax_stars], columns=['objid'])
    big_df = tmp_df.copy()
    big_df['pm_over_error'] = gdat['pm'][parallax_stars]/gdat['pm_unc'][parallax_stars]
    big_df['parallax_over_error'] = gdat['parallax_over_error'][parallax_stars]
    big_df['g_mag'] = gdat['phot_g_mean_mag'][parallax_stars]
    
    print(file_num, gf.split("/")[-1], len(gdat[parallax_stars]), 
          len(parallax_stars[0])/len(gdat))
    if file_num == 0:
        master_df = tmp_df.copy()
        master_big = big_df.copy()
    else:
        master_df = master_df.append(tmp_df, ignore_index=True)
        master_big = master_big.append(big_df, ignore_index=True)

0 tmp_dec_0-result.fits 1541427 0.08401971305377691
1 tmp_dec_10-result.fits 1625552 0.08132591165782033
2 tmp_dec_15-result.fits 1631505 0.08467451946254603
3 tmp_dec_20-result.fits 1626780 0.09482543348719631
4 tmp_dec_25-result.fits 1736175 0.10259520803395512
5 tmp_dec_30-result.fits 1869020 0.11375738068430084
6 tmp_dec_35-result.fits 1893547 0.12038728202718123
7 tmp_dec_40-result-2.fits 1823504 0.12614325941292287
8 tmp_dec_40-result.fits 1823504 0.12614325941292287
9 tmp_dec_45-result.fits 2037387 0.12949332600461091
10 tmp_dec_5-result.fits 1622216 0.08400522958725938
11 tmp_dec_50-result.fits 1956138 0.126298813533474
12 tmp_dec_55-result.fits 1742525 0.12716014994538943
13 tmp_dec_60-result.fits 1395872 0.12948200962633039
14 tmp_dec_65-result.fits 939809 0.14283234837506584
15 tmp_dec_70-result.fits 751172 0.17534073906355405
16 tmp_dec_75-result.fits 507336 0.2240287697352559
17 tmp_dec_80-result.fits 298657 0.2559819869702094
18 tmp_dec_85-result.fits 92470 0.297002672285

In [6]:
master_df.to_hdf(gaia_dir + "parallax_objid_stars.h5", "d1")
len(master_df)

38764553

In [7]:
master_big.to_hdf(gaia_dir + "parallax_ps1_gaia_mag_pm_plx.h5", "d1")
len(master_big)

38764553

In [11]:
for file_num, gf in enumerate(gaia_files):
    gdat = fits.getdata(gf)
    parallax_stars = np.where( (gdat['parallax_over_error'] >= 8) & 
                               (gdat['number_of_neighbours'] == 1) & 
                               (gdat['number_of_mates'] == 0))
    pm_stars = np.where((gdat['pm']/gdat['pm_unc'] >= 7.5) &
                         (gdat['number_of_neighbours'] == 1) & 
                         (gdat['number_of_mates'] == 0))
    tmp_df = pd.DataFrame(gdat['original_ext_source_id'][pm_stars], columns=['objid'])
    big_df = tmp_df.copy()
    big_df['pm_over_error'] = gdat['pm'][pm_stars]/gdat['pm_unc'][pm_stars]
    big_df['parallax_over_error'] = gdat['parallax_over_error'][pm_stars]
    big_df['g_mag'] = gdat['phot_g_mean_mag'][pm_stars]
    
    print(file_num, gf.split("/")[-1], len(gdat[pm_stars]), 
          len(pm_stars[0])/len(gdat))
    if file_num == 0:
        master_df = tmp_df.copy()
        master_big = big_df.copy()
    else:
        master_df = master_df.append(tmp_df, ignore_index=True)
        master_big = master_big.append(big_df, ignore_index=True)

0 tmp_dec_0-result.fits 10291179 0.5609489820569218
1 tmp_dec_10-result.fits 12190235 0.6098740456768343
2 tmp_dec_15-result.fits 12248974 0.635717320730994
3 tmp_dec_20-result.fits 11731368 0.6838245220606494
4 tmp_dec_25-result.fits 11732292 0.6932924033896971
5 tmp_dec_30-result.fits 11394366 0.6935149065918258
6 tmp_dec_35-result.fits 10473087 0.6658543349408308
7 tmp_dec_40-result-2.fits 8846825 0.6119906185869246
8 tmp_dec_40-result.fits 8846825 0.6119906185869246
9 tmp_dec_45-result.fits 9645035 0.6130242617533549
10 tmp_dec_5-result.fits 11133079 0.5765180823072242
11 tmp_dec_50-result.fits 8906812 0.5750717934857912
12 tmp_dec_55-result.fits 7518492 0.5486593134005026
13 tmp_dec_60-result.fits 5775324 0.5357228727012053
14 tmp_dec_65-result.fits 3417087 0.5193295242032249
15 tmp_dec_70-result.fits 2357200 0.5502244361086537
16 tmp_dec_75-result.fits 1415996 0.6252736683973608
17 tmp_dec_80-result.fits 774356 0.6637084933629664
18 tmp_dec_85-result.fits 218622 0.702187933603987

In [12]:
master_df.to_hdf(gaia_dir + "pm_objid_stars.h5", "d1")
len(master_df)

234176264

In [13]:
master_big.to_hdf(gaia_dir + "pm_ps1_gaia_mag_pm_plx.h5", "d1")
len(master_big)

234176264